# Code Testing with `pytest` and `doctest`

David Orme

## What is code testing?

* How do you know your code does what it claims to do?
    * "Well, that's what I wrote it to do"
    * "Well, if I give it this inputs it does this, **as expected**"
* Testing frameworks allow us to:
    * Give inputs to code 
    * Check that it does what is expected
    * Repeatably and easily
    * A vital part of **continuous integration**

## Continuous integration

* Code from a team is pushed into `develop` regularly.
* Need to be aware of **breaking changes**:
    * Updated algorithm gives different answers
    * Updated code takes different arguments
* Code testing helps catch breaking changes:
    * Catch and fix code errors
    * Need to release new major version

## Semantic versioning

* Strict rules for version numbers
    * https://semver.org/
    * https://github.com/semver/semver
* With MAJOR.MINOR.PATCH, increment the:
    * MAJOR version when you make incompatible API changes,
    * MINOR version when you add functionality in a backwards compatible manner, and
    * PATCH version when you make backwards compatible bug fixes.



## What should tests be?

* Pieces of code
* Expected outcomes of running code
* Individually small in scope
* Not just testing the **right answer**
* Test failure modes
* Quick to run

## Frameworks

* Lots of options, including:
    * [The `pytest` extension](https://docs.pytest.org/)
    * [The `doctest` standard library](https://docs.python.org/3/library/doctest.html)
    * [The `unittest` standard library](https://docs.python.org/3/library/unittest.html)
    * [The `nose` extension](https://nose.readthedocs.io/en/latest/)

## The `doctest` package

* The **docstrings** for code often includes example use
* The `doctest` package runs those examples
* Check if documented example code is **correct**
    * Looks for `>>>`, `+++` and output
    * Typically simple, self-contained tests
    * **Exact match** to expected output



## The `doctest` package

In [32]:
# %load -s my_float_multiplier mfm.py
def my_float_multiplier(x: float, y: float) -> float:
    """Multiplies two floats together.

    Arguments:
        x: The first number
        y: The second number

    Examples:
        >>> my_float_multiplier(2.1, 3.6)
        7.56
    """

    return x * y


## The `doctest` package

```bash
dorme@MacBook-Pro training % python -m doctest mfm.py
*****************************************************
File "mfm.py", line 9, in mfm.my_float_multiplier
Failed example:
    my_float_multiplier(2.1, 3.6)
Expected:
    7.56
Got:
    7.5600000000000005
*****************************************************
1 items had failures:
   1 of   1 in untitled.my_float_multiplier
***Test Failed*** 1 failures.
dorme@MacBook-Pro training % 
```

In [33]:
# %%bash
# This would be neater, but have to hack to suppress the 
# non-zero exit status from doctest
# python -m doctest mfm.py || true


## The `doctest` package

* Adjust the expected value

In [42]:
# %load -s my_float_multiplier2 mfm.py
def my_float_multiplier2(x: float, y: float) -> float:
    """Multiplies two floats together.

    Arguments:
        x: The first number
        y: The second number

    Examples:
        >>> round(my_float_multiplier(2.1, 3.6), 2)
        7.56
    """


## The `doctest` package

* Use `doctest` **directives**

In [41]:
# %load -s my_float_multiplier3 mfm.py
def my_float_multiplier3(x: float, y: float) -> float:
    """Multiplies two floats together.

    Arguments:
        x: The first number
        y: The second number

    Examples:
        >>> my_float_multiplier(2.1, 3.6) #doctest: +ELLIPSIS
        7.56...
    """


## The `doctest` package

* A more complex example

In [37]:
# %load -s TimesTable mfm.py
class TimesTable:
    """Create times tables for a number.

    The TimesTable instance can be used to produce a times
    table for a specific number.

    Attributes:
        num: The base number to use for tables

    Args:
        num: Sets the `num` attribute

    Examples:
        >>> seven_tt = TimesTable(num = 7)
        >>> seven_tt  # doctest: +ELLIPSIS
        <mfm.TimesTable object at 0x...>
        >>> seven_tt.num
        7
    """

    def __init__(self, num: int) -> None:

        self.num = num

    def table(self, start: int = 1, stop: int = 10) -> List[int]:
        """Calculate a table.

        Returns a times table for the initialised base number from
        start to stop.

        Args:
            start: Start number for the table
            stop: End number for the table

        Examples:
            >>> seven_tt = TimesTable(num = 7)
            >>> seven_tt.table(2,7)
            [14, 21, 28, 35, 42, 49]
        """

        return [self.num * v for v in range(start, stop + 1)]


## The `doctest` package

* Invaluable for maintaining documentation quality
* Spot checks on code function
* Limited to relatively simple uses
* Tests for each docstring run **independently**

## The `pytest` framework

* The main testing framework
* Much more powerful, also more complex!
* Tests in a separate directory
* Running `pytest` automatically finds tests in files and runs them


## The `pytest` framework

* Directory structure:
    * A separate directory in the package root (`test`)
    * Can contain subfolders (e.g. grouped by package module)
    * Contains test files: `test_xyz.py`
* Test files contain: 
    * classes (e.g. `TestClass`)
    * functions (e.g. `test_function`)



## A `pytest` test

* A really simple example
* A function that runs a bit of code
* And **`asserts`** that the output equals a value

In [47]:
# %load -s test_my_float_multiplier test_mfm.py
def test_my_float_multiplier():
    
    assert 10 == my_float_multiplier(2,5)


## Testing failure modes

* A more picky function

In [51]:
# %load -s my_picky_float_multiplier mfm.py
def my_picky_float_multiplier(x: float, y: float) -> float:
    """Multiplies two floats together.

    Arguments:
        x: The first number
        y: The second number

    Examples:
        >>> my_float_multiplier3(2.1, 3.6) #doctest: +ELLIPSIS
        7.56...
    """

    if not (isinstance(x, float) and isinstance(y, float)):
        raise ValueError('Both x and y must be of type float')
    
    return x * y


## Testing failure modes

* Use the `pytest.raises` context manager
* Traps exceptions
* Makes sure the exception is the right type

In [53]:
# %load -s test_my_picky_float_multiplier test_mfm.py
def test_my_picky_float_multiplier():
    
    with pytest.raises(ValueError) as err_hndlr:
        
         x = my_picky_float_multiplier(2,5)
            
    assert str(err_hndlr.value) == "Both x and y must be of type float"


In [84]:
%%bash
pytest -v test_mfm.py::test_fm test_mfm.py::test_pfm test_mfm.py::test_pfm_fail || true

============================= test session starts ==============================
platform darwin -- Python 3.7.7, pytest-6.1.2, py-1.9.0, pluggy-0.13.1 -- /Users/dorme/.pyenv/versions/3.7.7/bin/python3.7
cachedir: .pytest_cache
rootdir: /Users/dorme/Research/Virtual_Rainforest/virtual_rainforest/source/development/training
plugins: anyio-3.6.1, pyfakefs-4.5.3, typeguard-2.13.3
collecting ... collected 3 items

test_mfm.py::test_fm PASSED                                              [ 33%]
test_mfm.py::test_pfm PASSED                                             [ 66%]
test_mfm.py::test_pfm_fail FAILED                                        [100%]

=================================== FAILURES ===================================
________________________________ test_pfm_fail _________________________________

    def test_pfm_fail():
    
        with pytest.raises(ValueError) as err_hndlr:
    
>           x = my_picky_float_multiplier(2.0, 5.0)
E           Failed: DID NOT RAISE <class '

## Parameterization

* Reuse the same test function
* Test different input values

In [75]:
# %load -r 3,22-33 test_mfm.py
import pytest
@pytest.mark.parametrize(
    argnames=["x", "y", "expected"],
    argvalues=[
        (1.0, 3.0, 3.0),
        (1.5, -3.0, -4.5),
        (-1.5, 3.0, -4.5),
        (-1.5, -3.0, 4.5),
    ],
)
def test_pfm_param_noid(x, y, expected):

    assert expected == my_picky_float_multiplier(x, y)

# Running `pytest`


In [71]:
%%bash
pytest -v -k "not ids"

============================= test session starts ==============================
platform darwin -- Python 3.7.7, pytest-6.1.2, py-1.9.0, pluggy-0.13.1 -- /Users/dorme/.pyenv/versions/3.7.7/bin/python3.7
cachedir: .pytest_cache
rootdir: /Users/dorme/Research/Virtual_Rainforest/virtual_rainforest/source/development/training
plugins: anyio-3.6.1, pyfakefs-4.5.3, typeguard-2.13.3
collecting ... collected 10 items / 4 deselected / 6 selected

test_mfm.py::test_fm PASSED                                              [ 16%]
test_mfm.py::test_pfm PASSED                                             [ 33%]
test_mfm.py::test_pfm_param_noid[1.0-3.0-3.0] PASSED                     [ 50%]
test_mfm.py::test_pfm_param_noid[1.5--3.0--4.5] PASSED                   [ 66%]
test_mfm.py::test_pfm_param_noid[-1.5-3.0--4.5] PASSED                   [ 83%]
test_mfm.py::test_pfm_param_noid[-1.5--3.0-4.5] PASSED                   [100%]

======================= 6 passed, 4 deselected in 0.02s ====================

## Parameterization: test ids

* Adds a label for each parameterized test

In [80]:
# %load -r 36-48 test_mfm.py
@pytest.mark.parametrize(
    argnames=["x", "y", "expected"],
    argvalues=[
        (1.0, 3.0, 3.0),
        (1.5, -3.0, -4.5),
        (-1.5, 3.0, -4.5),
        (-1.5, -3.0, 4.5),
    ],
    ids=["++", "-+", "+-", "--"],
)
def test_pfm_param_ids(x, y, expected):

    assert expected == my_picky_float_multiplier(x, y)

# Running `pytest`


In [79]:
%%bash
pytest -v -k "not noid"

============================= test session starts ==============================
platform darwin -- Python 3.7.7, pytest-6.1.2, py-1.9.0, pluggy-0.13.1 -- /Users/dorme/.pyenv/versions/3.7.7/bin/python3.7
cachedir: .pytest_cache
rootdir: /Users/dorme/Research/Virtual_Rainforest/virtual_rainforest/source/development/training
plugins: anyio-3.6.1, pyfakefs-4.5.3, typeguard-2.13.3
collecting ... collected 10 items / 4 deselected / 6 selected

test_mfm.py::test_fm PASSED                                              [ 16%]
test_mfm.py::test_pfm PASSED                                             [ 33%]
test_mfm.py::test_pfm_param_ids[++] PASSED                               [ 50%]
test_mfm.py::test_pfm_param_ids[-+] PASSED                               [ 66%]
test_mfm.py::test_pfm_param_ids[+-] PASSED                               [ 83%]
test_mfm.py::test_pfm_param_ids[--] PASSED                               [100%]

======================= 6 passed, 4 deselected in 0.02s ====================

## Fixtures

* Reusable bits of code shared between tests
* Modular desi